In [119]:
from playwright.async_api import async_playwright
import asyncio

from datetime import datetime, timedelta

In [121]:
async def fetch_with_playwright():
    news_list = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        
        page = await browser.new_page()
        await page.set_extra_http_headers({
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"
        })
        await page.goto("https://www.prothomalo.com/collection/latest", timeout=60000)
        await page.wait_for_selector("#container", timeout=15000)
        #Wait for a card to appear inside container
        await page.wait_for_selector("#container div.serial-stories-tabs", timeout=10000)
        #page.locator('h2.tab-headline', has_text = 'পঠিত').click()
        #page.wait_for_load_state("networkidle")
        await page.wait_for_selector("div.numbered-story-headline")
        await page.evaluate("""
            () => {
                const tabs = Array.from(document.querySelectorAll('h2.tab-headline'));
                const target = tabs.find(tab => tab.innerText.includes('পঠিত'));
                if (target) target.click();
            }
        """)
        elements = await page.locator("div.numbered-story-headline").all()
        news_element = []
        seen_urls = set()
        i = 1;
        for el in elements:
            link_el  =  el.locator("a.title-link")
            href = await link_el.get_attribute("href")
            title = (await el.inner_text()).strip()
            if href not in seen_urls:
                news_element.append({
                    "portal_rank": i,
                    "title": title,
                    "href": href,
                    "script_at": datetime.now()
                })
                seen_urls.add(href)
                i = i + 1

        if len(news_element) > 0:
            for element in news_element:
                detail_page = await browser.new_page()
                await detail_page.set_extra_http_headers({
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"
                })
                print(element["href"])
                await detail_page.goto(element["href"], timeout = 20000)
                # await page.wait_for_selector("div.story-head")
                await detail_page.wait_for_selector("div.story-head", state="attached", timeout=60000)
                elements_details = await detail_page.locator("div.story-content").all()
                for item in elements_details:
                    text = await item.inner_text()
                    print(text)
                #link_el  =  elements_details.locator("a.title-link")
                #print(elements_details)
                img_elements = await detail_page.locator("img").all()
                # Extract the src attribute from each
                image_urls = [await img.get_attribute("src") for img in img_elements if await img.get_attribute("src")]
                # Print or use as needed
                for url in image_urls:
                    print(url)
                    
                await detail_page.close()
                
                    
        await browser.close()
        return news_element
            
            #print('title', title, 'href', href, 'link_el', link_el)
        #    # Get the nearest ancestor or surrounding time tag with class 'published-time'
        #    time_locator = el.locator("xpath=following::time[contains(@class, 'published-time')]")
        
            # Check if it exists
       #     if await time_locator.count() > 0:
       #         published_time = await time_locator.first.inner_text()
       #     else:
       #         published_time = "No time found"
       #     print("published_time", published_time, title)
       #     news_list.append((title, href, published_time))
       #     
       # await browser.close()
    #return news_list
        #for i in range(articles.count())

In [123]:
await fetch_with_playwright()

https://www.prothomalo.com/politics/tuomam7a71
প্রধান উপদেষ্টার সঙ্গে বৈঠক শেষে সাংবাদিকদের মুখোমুখি বিএনপির মহাসচিব মির্জা ফখরুল ইসলাম আলমগীরসহ দলটির নেতারাছবি: প্রথম আলো

প্রধান উপদেষ্টা অধ্যাপক মুহাম্মদ ইউনূসের সঙ্গে আলোচনায় বিএনপি একেবারেই সন্তুষ্ট নয়। প্রধান উপদেষ্টার সঙ্গে বৈঠকের পর এ কথা বলেছেন বিএনপির মহাসচিব মির্জা ফখরুল ইসলাম আলমগীর।

আজ বুধবার দুপুরে রাষ্ট্রীয় অতিথি ভবন যমুনায় অন্তর্বর্তী সরকারের প্রধান উপদেষ্টা অধ্যাপক ইউনূসের সঙ্গে প্রায় দুই ঘণ্টা বৈঠক করে বিএনপির প্রতিনিধিদল। বিএনপির মহাসচিব মির্জা ফখরুল দলটির সাত সদস্যের এই প্রতিনিধিদলের নেতৃত্ব দেন।

প্রধান উপদেষ্টা অধ্যাপক মুহাম্মদ ইউনূসের সঙ্গে বৈঠকে বিএনপির মহাসচিব মির্জা ফখরুল ইসলাম আলমগীরছবি: বিএনপির ফেসবুক পেজ

বৈঠক শেষে মির্জা ফখরুল সাংবাদিকদের বলেন, প্রধান উপদেষ্টা কোনো সুনির্দিষ্ট ডেডলাইন তাঁদের দেননি। তিনি ডিসেম্বর থেকে জুনের মধ্যে নির্বাচনের কথা বলেছেন।

মির্জা ফখরুল বলেন, ‘আমরা একেবারেই সন্তুষ্ট নই। আমরা পরিষ্কার করে বলেছি, ডিসেম্বর মধ্যে নির্বাচন না হলে দেশের রাজনৈতিক, অর্থনৈতিক ও সামাজিক পরিস্থিতি আরও খারা

[{'portal_rank': 1,
  'title': '১\nপ্রধান উপদেষ্টার সঙ্গে বৈঠক আমরা একেবারেই সন্তুষ্ট নই: মির্জা ফখরুল',
  'href': 'https://www.prothomalo.com/politics/tuomam7a71',
  'script_at': datetime.datetime(2025, 4, 17, 16, 19, 45, 20209)},
 {'portal_rank': 2,
  'title': '২\nযে বিরল খনিজ নিয়ে যুক্তরাষ্ট্র-চীন দ্বন্দ্ব, তা পাওয়ার সম্ভাবনা বাংলাদেশে',
  'href': 'https://www.prothomalo.com/bangladesh/444c8vs5xm',
  'script_at': datetime.datetime(2025, 4, 17, 16, 19, 45, 86213)},
 {'portal_rank': 3,
  'title': '৩\nরাজনীতিতে আসা ভুল ছিল না, নির্বাচন করলে আবার জিতব: সাকিব',
  'href': 'https://www.prothomalo.com/sports/cricket/tikzktkakp',
  'script_at': datetime.datetime(2025, 4, 17, 16, 19, 45, 152218)},
 {'portal_rank': 4,
  'title': '৪\n১৮ বছর নাকি ৬ মাস, কোনটা দেখে আমাকে বিচার করবেন তা আপনাদের ইচ্ছা: সাকিব আল হাসান',
  'href': 'https://www.prothomalo.com/sports/cricket/y24hphtilt',
  'script_at': datetime.datetime(2025, 4, 17, 16, 19, 45, 212224)},
 {'portal_rank': 5,
  'title': '৫\nতাঁর স্মৃতি দ